# Colab FAQ

For some basic overview and features offered in Colab notebooks, check out: [Overview of Colaboratory Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb)

You need to use the colab GPU for this assignmentby selecting:

> **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**

## Setup PyTorch
All files are stored at /content/csc421/a4/ folder


In [23]:
######################################################################
# Setup python environment and change the current working directory
######################################################################
!pip install torch torchvision
!pip install imageio

!pip install matplotlib

%mkdir -p /content/csc421/a4/
%cd /content/csc421/a4

/content/csc421/a4


# Helper code

## Utility functions

In [0]:
import os

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from six.moves.urllib.request import urlretrieve
import tarfile

import imageio
from urllib.error import URLError
from urllib.error import HTTPError


def get_file(fname,
             origin,
             untar=False,
             extract=False,
             archive_format='auto',
             cache_dir='data'):
    datadir = os.path.join(cache_dir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + '.tar.gz'
    else:
        fpath = os.path.join(datadir, fname)

    print(fpath)
    if not os.path.exists(fpath):
        print('Downloading data from', origin)

        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                urlretrieve(origin, fpath)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

    if untar:
        if not os.path.exists(untar_fpath):
            print('Extracting file.')
            with tarfile.open(fpath) as archive:
                archive.extractall(datadir)
        return untar_fpath

    return fpath


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

                
def to_var(tensor, cuda=True):
    """Wraps a Tensor in a Variable, optionally placing it on the GPU.

        Arguments:
            tensor: A Tensor object.
            cuda: A boolean flag indicating whether to use the GPU.

        Returns:
            A Variable object, on the GPU if cuda==True.
    """
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

    
def to_data(x):
    """Converts variable to numpy."""
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()


def create_dir(directory):
    """Creates a directory if it doesn't already exist.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)


def gan_checkpoint(iteration, G, D, opts):
    """Saves the parameters of the generator G and discriminator D.
    """
    G_path = os.path.join(opts.checkpoint_dir, 'G.pkl')
    D_path = os.path.join(opts.checkpoint_dir, 'D.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)


def cyclegan_checkpoint(iteration, G_XtoY, G_YtoX, D_X, D_Y, opts):
    """Saves the parameters of both generators G_YtoX, G_XtoY and discriminators D_X, D_Y.
    """
    G_XtoY_path = os.path.join(opts.checkpoint_dir, 'G_XtoY.pkl')
    G_YtoX_path = os.path.join(opts.checkpoint_dir, 'G_YtoX.pkl')
    D_X_path = os.path.join(opts.checkpoint_dir, 'D_X.pkl')
    D_Y_path = os.path.join(opts.checkpoint_dir, 'D_Y.pkl')
    torch.save(G_XtoY.state_dict(), G_XtoY_path)
    torch.save(G_YtoX.state_dict(), G_YtoX_path)
    torch.save(D_X.state_dict(), D_X_path)
    torch.save(D_Y.state_dict(), D_Y_path)


def load_checkpoint(opts):
    """Loads the generator and discriminator models from checkpoints.
    """
    G_XtoY_path = os.path.join(opts.load, 'G_XtoY.pkl')
    G_YtoX_path = os.path.join(opts.load, 'G_YtoX.pkl')
    D_X_path = os.path.join(opts.load, 'D_X.pkl')
    D_Y_path = os.path.join(opts.load, 'D_Y.pkl')

    G_XtoY = CycleGenerator(conv_dim=opts.g_conv_dim, init_zero_weights=opts.init_zero_weights)
    G_YtoX = CycleGenerator(conv_dim=opts.g_conv_dim, init_zero_weights=opts.init_zero_weights)
    D_X = DCDiscriminator(conv_dim=opts.d_conv_dim)
    D_Y = DCDiscriminator(conv_dim=opts.d_conv_dim)

    G_XtoY.load_state_dict(torch.load(G_XtoY_path, map_location=lambda storage, loc: storage))
    G_YtoX.load_state_dict(torch.load(G_YtoX_path, map_location=lambda storage, loc: storage))
    D_X.load_state_dict(torch.load(D_X_path, map_location=lambda storage, loc: storage))
    D_Y.load_state_dict(torch.load(D_Y_path, map_location=lambda storage, loc: storage))

    if torch.cuda.is_available():
        G_XtoY.cuda()
        G_YtoX.cuda()
        D_X.cuda()
        D_Y.cuda()
        print('Models moved to GPU.')

    return G_XtoY, G_YtoX, D_X, D_Y


def merge_images(sources, targets, opts):
    """Creates a grid consisting of pairs of columns, where the first column in
    each pair contains images source images and the second column in each pair
    contains images generated by the CycleGAN from the corresponding images in
    the first column.
    """
    _, _, h, w = sources.shape
    row = int(np.sqrt(opts.batch_size))
    merged = np.zeros([3, row * h, row * w * 2])
    for (idx, s, t) in (zip(range(row ** 2), sources, targets, )):
        i = idx // row
        j = idx % row
        merged[:, i * h:(i + 1) * h, (j * 2) * h:(j * 2 + 1) * h] = s
        merged[:, i * h:(i + 1) * h, (j * 2 + 1) * h:(j * 2 + 2) * h] = t
    return merged.transpose(1, 2, 0)


def generate_gif(directory_path, keyword=None):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".png") and (keyword is None or keyword in filename):
            img_path = os.path.join(directory_path, filename)
            print("adding image {}".format(img_path))
            images.append(imageio.imread(img_path))

    if keyword:
        imageio.mimsave(
            os.path.join(directory_path, 'anim_{}.gif'.format(keyword)), images)
    else:
        imageio.mimsave(os.path.join(directory_path, 'anim.gif'), images)


def create_image_grid(array, ncols=None):
    """
    """
    num_images, channels, cell_h, cell_w = array.shape
    if not ncols:
        ncols = int(np.sqrt(num_images))
    nrows = int(np.math.floor(num_images / float(ncols)))
    result = np.zeros((cell_h * nrows, cell_w * ncols, channels), dtype=array.dtype)
    for i in range(0, nrows):
        for j in range(0, ncols):
            result[i * cell_h:(i + 1) * cell_h, j * cell_w:(j + 1) * cell_w, :] = array[i * ncols + j].transpose(1, 2,
                                                                                                                 0)

    if channels == 1:
        result = result.squeeze()
    return result


def gan_save_samples(G, fixed_noise, iteration, opts):
    generated_images = G(fixed_noise)
    generated_images = to_data(generated_images)

    grid = create_image_grid(generated_images)

    # merged = merge_images(X, fake_Y, opts)
    path = os.path.join(opts.sample_dir, 'sample-{:06d}.png'.format(iteration))
    imageio.imwrite(path, grid)
    print('Saved {}'.format(path))


def cyclegan_save_samples(iteration, fixed_Y, fixed_X, G_YtoX, G_XtoY, opts):
    """Saves samples from both generators X->Y and Y->X.
    """
    fake_X = G_YtoX(fixed_Y)
    fake_Y = G_XtoY(fixed_X)

    X, fake_X = to_data(fixed_X), to_data(fake_X)
    Y, fake_Y = to_data(fixed_Y), to_data(fake_Y)

    merged = merge_images(X, fake_Y, opts)
    path = os.path.join(opts.sample_dir, 'sample-{:06d}-X-Y.png'.format(iteration))
    imageio.imwrite(path, merged)
    print('Saved {}'.format(path))

    merged = merge_images(Y, fake_X, opts)
    path = os.path.join(opts.sample_dir, 'sample-{:06d}-Y-X.png'.format(iteration))
    imageio.imwrite(path, merged)
    print('Saved {}'.format(path))

## Data loader

In [0]:
def get_emoji_loader(emoji_type, opts):
    """Creates training and test data loaders.
    """
    transform = transforms.Compose([
                    transforms.Scale(opts.image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

    train_path = os.path.join('data/emojis', emoji_type)
    test_path = os.path.join('data/emojis', 'Test_{}'.format(emoji_type))

    train_dataset = datasets.ImageFolder(train_path, transform)
    test_dataset = datasets.ImageFolder(test_path, transform)

    train_dloader = DataLoader(dataset=train_dataset, batch_size=opts.batch_size, shuffle=True, num_workers=opts.num_workers)
    test_dloader = DataLoader(dataset=test_dataset, batch_size=opts.batch_size, shuffle=False, num_workers=opts.num_workers)

    return train_dloader, test_dloader

## Training and evaluation code

In [0]:
def print_models(G_XtoY, G_YtoX, D_X, D_Y):
    """Prints model information for the generators and discriminators.
    """
    if G_YtoX:
        print("                 G_XtoY                ")
        print("---------------------------------------")
        print(G_XtoY)
        print("---------------------------------------")

        print("                 G_YtoX                ")
        print("---------------------------------------")
        print(G_YtoX)
        print("---------------------------------------")

        print("                  D_X                  ")
        print("---------------------------------------")
        print(D_X)
        print("---------------------------------------")

        print("                  D_Y                  ")
        print("---------------------------------------")
        print(D_Y)
        print("---------------------------------------")
    else:
        print("                 G                     ")
        print("---------------------------------------")
        print(G_XtoY)
        print("---------------------------------------")

        print("                  D                    ")
        print("---------------------------------------")
        print(D_X)
        print("---------------------------------------")


def create_model(opts):
    """Builds the generators and discriminators.
    """
    if opts.Y is None:
        ### GAN
        G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
        D = DCDiscriminator(conv_dim=opts.d_conv_dim, spectral_norm=opts.spectral_norm)

        print_models(G, None, D, None)

        if torch.cuda.is_available():
            G.cuda()
            D.cuda()
            print('Models moved to GPU.')
        return G, D
          
    else:
        ### CycleGAN
        G_XtoY = CycleGenerator(conv_dim=opts.g_conv_dim, init_zero_weights=opts.init_zero_weights)
        G_YtoX = CycleGenerator(conv_dim=opts.g_conv_dim, init_zero_weights=opts.init_zero_weights)
        D_X = DCDiscriminator(conv_dim=opts.d_conv_dim)
        D_Y = DCDiscriminator(conv_dim=opts.d_conv_dim)

        print_models(G_XtoY, G_YtoX, D_X, D_Y)

        if torch.cuda.is_available():
            G_XtoY.cuda()
            G_YtoX.cuda()
            D_X.cuda()
            D_Y.cuda()
            print('Models moved to GPU.')
        return G_XtoY, G_YtoX, D_X, D_Y


def train(opts):
    """Loads the data, creates checkpoint and sample directories, and starts the training loop.
    """

    # Create train and test dataloaders for images from the two domains X and Y
    dataloader_X, test_dataloader_X = get_emoji_loader(emoji_type=opts.X, opts=opts)
    if opts.Y:
        dataloader_Y, test_dataloader_Y = get_emoji_loader(emoji_type=opts.Y, opts=opts)

    # Create checkpoint and sample directories
    create_dir(opts.checkpoint_dir)
    create_dir(opts.sample_dir)

    # Start training
    if opts.Y is None:
        G, D = gan_training_loop(dataloader_X, test_dataloader_X, opts)
        return G, D
    else:
        G_XtoY, G_YtoX, D_X, D_Y = cyclegan_training_loop(dataloader_X, dataloader_Y, test_dataloader_X, test_dataloader_Y, opts)
        return G_XtoY, G_YtoX, D_X, D_Y


def print_opts(opts):
    """Prints the values of all command-line arguments.
    """
    print('=' * 80)
    print('Opts'.center(80))
    print('-' * 80)
    for key in opts.__dict__:
        if opts.__dict__[key]:
            print('{:>30}: {:<30}'.format(key, opts.__dict__[key]).center(80))
    print('=' * 80)


# Your code for generators and discriminators

## Helper modules

In [0]:
def sample_noise(batch_size, dim):
    """
    Generate a PyTorch Tensor of uniform random noise.

    Input:
    - batch_size: Integer giving the batch size of noise to generate.
    - dim: Integer giving the dimension of noise to generate.

    Output:
    - A PyTorch Tensor of shape (batch_size, dim, 1, 1) containing uniform
      random noise in the range (-1, 1).
    """
    return to_var(torch.rand(batch_size, dim) * 2 - 1).unsqueeze(2).unsqueeze(3)
  

def upconv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, spectral_norm=False):
    """Creates a upsample-and-convolution layer, with optional batch normalization.
    """
    layers = []
    if stride>1:
        layers.append(nn.Upsample(scale_factor=stride))
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=False)
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)


def conv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, init_zero_weights=False, spectral_norm=False):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    if init_zero_weights:
        conv_layer.weight.data = torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.001
            
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)
  

class ResnetBlock(nn.Module):
    def __init__(self, conv_dim):
        super(ResnetBlock, self).__init__()
        self.conv_layer = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = x + self.conv_layer(x)
        return out

## DCGAN

## Spectral Norm class

In [0]:
def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False

    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)

    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)

### GAN generator

In [0]:
class DCGenerator(nn.Module):
    def __init__(self, noise_size, conv_dim, spectral_norm=False):
        super(DCGenerator, self).__init__()

        self.conv_dim = conv_dim
        ###########################################
        ##   FILL THIS IN: CREATE ARCHITECTURE   ##
        ###########################################
        self.linear_bn = upconv(noise_size, self.conv_dim*4, kernel_size=5, stride=4)
        self.upconv1 = upconv(self.conv_dim*4, self.conv_dim*2, kernel_size=5, stride=2)
        self.upconv2 = upconv(self.conv_dim*2, self.conv_dim, kernel_size=5, stride=2)
        self.upconv3 = upconv(self.conv_dim, 3, kernel_size=5, stride=2, batch_norm=False)

    def forward(self, z):
        """Generates an image given a sample of random noise.

            Input
            -----
                z: BS x noise_size x 1 x 1   -->  BSx100x1x1 (during training)

            Output
            ------
                out: BS x channels x image_width x image_height  -->  BSx3x32x32 (during training)
        """
        batch_size = z.size(0)
        
        out = F.relu(self.linear_bn(z)).view(-1, self.conv_dim*4, 4, 4)    # BS x 128 x 4 x 4
        out = F.relu(self.upconv1(out))  # BS x 64 x 8 x 8
        out = F.relu(self.upconv2(out))  # BS x 32 x 16 x 16
        out = F.tanh(self.upconv3(out))  # BS x 3 x 32 x 32
        
        out_size = out.size()
        if out_size != torch.Size([batch_size, 3, 32, 32]):
            raise ValueError("expect {} x 3 x 32 x 32, but get {}".format(batch_size, out_size))
        return out


### GAN discriminator

In [0]:
class DCDiscriminator(nn.Module):
    """Defines the architecture of the discriminator network.
       Note: Both discriminators D_X and D_Y have the same architecture in this assignment.
    """
    def __init__(self, conv_dim=64, spectral_norm=False):
        super(DCDiscriminator, self).__init__()

        ###########################################
        ##   FILL THIS IN: CREATE ARCHITECTURE   ##
        ###########################################

        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv4 = conv(in_channels=conv_dim*4, out_channels=1, kernel_size=5, stride=2, padding=1, batch_norm=False, spectral_norm=spectral_norm)

    def forward(self, x):
        batch_size = x.size(0)

        out = F.relu(self.conv1(x))    # BS x 64 x 16 x 16
        out = F.relu(self.conv2(out))    # BS x 64 x 8 x 8
        out = F.relu(self.conv3(out))    # BS x 64 x 4 x 4

        out = self.conv4(out).squeeze()
        out_size = out.size()
        if out_size != torch.Size([batch_size,]):
            raise ValueError("expect {} x 1, but get {}".format(batch_size, out_size))
        return out

### GAN training loop

In [0]:
def gan_training_loop(dataloader, test_dataloader, opts):
    """Runs the training loop.
        * Saves checkpoint every opts.checkpoint_every iterations
        * Saves generated samples every opts.sample_every iterations
    """

    # Create generators and discriminators
    G, D = create_model(opts)

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = optim.Adam(d_params, opts.lr * 2., [opts.beta1, opts.beta2])

    train_iter = iter(dataloader)

    test_iter = iter(test_dataloader)

    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_noise = sample_noise(100, opts.noise_size)  # # 100 x noise_size x 1 x 1

    iter_per_epoch = len(train_iter)
    total_train_iters = opts.train_iters

    losses = {"iteration": [], "D_fake_loss": [], "D_real_loss": [], "G_loss": []}

    gp_weight = 10

    try:
        for iteration in range(1, opts.train_iters + 1):

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
                train_iter = iter(dataloader)

            real_images, real_labels = train_iter.next()
            real_images, real_labels = to_var(real_images), to_var(real_labels).long().squeeze()

            # ones = Variable(torch.Tensor(real_images.shape[0]).float().cuda().fill_(1.0), requires_grad=False)

            for d_i in range(opts.d_train_iters):
                d_optimizer.zero_grad()

                # FILL THIS IN
                # 1. Compute the discriminator loss on real images
                D_real_loss = torch.mean((D(real_images) - 1)**2)/2

                # 2. Sample noise
                noise = sample_noise(100, opts.noise_size)

                # 3. Generate fake images from the noise
                fake_images = G(noise)
                
                # 4. Compute the discriminator loss on the fake images
                D_fake_loss = torch.mean(D(fake_images)**2)/2 

                # ---- Gradient Penalty ----
                if opts.gradient_penalty:
                    alpha = torch.rand(real_images.shape[0], 1, 1, 1)
                    alpha = alpha.expand_as(real_images).cuda()
                    interp_images = Variable(alpha * real_images.size(0) + (1 - alpha) * fake_images.size(0), requires_grad=True).cuda()
                    D_interp_output = D(interp_images)

                    gradients = torch.autograd.grad(outputs=D_interp_output, inputs=interp_images,
                                                    grad_outputs=torch.ones(D_interp_output.size()).cuda(),
                                                    create_graph=True, retain_graph=True)[0]
                    gradients = gradients.view(real_images.shape[0], -1)
                    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

                    gp = gp_weight * gradients_norm.mean()
                else:
                    gp = 0.0

                # --------------------------
                # 5. Compute the total discriminator loss
                D_total_loss = D_real_loss + D_fake_loss

                D_total_loss.backward()
                d_optimizer.step()

            ###########################################
            ###          TRAIN THE GENERATOR        ###
            ###########################################

            g_optimizer.zero_grad()

            # FILL THIS IN
            # 1. Sample noise
            noise = sample_noise(100, opts.noise_size)

            # 2. Generate fake images from the noise
            fake_images = G(noise)

            # 3. Compute the generator loss
            G_loss = torch.mean((D(fake_images) - 1)**2)

            G_loss.backward()
            g_optimizer.step()

            # Print the log info
            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_real_loss'].append(D_real_loss.item())
                losses['D_fake_loss'].append(D_fake_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_real_loss: {:6.4f} | D_fake_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                    iteration, total_train_iters, D_real_loss.item(), D_fake_loss.item(), G_loss.item()))

            # Save the generated samples
            if iteration % opts.sample_every == 0:
                gan_save_samples(G, fixed_noise, iteration, opts)

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                gan_checkpoint(iteration, G, D, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G, D

    plt.figure()
    plt.plot(losses['iteration'], losses['D_real_loss'], label='D_real')
    plt.plot(losses['iteration'], losses['D_fake_loss'], label='D_fake')
    plt.plot(losses['iteration'], losses['G_loss'], label='G')
    plt.legend()
    plt.savefig(os.path.join(opts.sample_dir, 'losses.png'))
    plt.close()
    return G, D

## CycleGAN

### CycleGAN generator

In [0]:
class CycleGenerator(nn.Module):
    """Defines the architecture of the generator network.
       Note: Both generators G_XtoY and G_YtoX have the same architecture in this assignment.
    """
    def __init__(self, conv_dim=64, init_zero_weights=False):
        super(CycleGenerator, self).__init__()

        # 1. Define the encoder part of the generator (that extracts features from the input image)
        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=5, init_zero_weights=init_zero_weights)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=5, init_zero_weights=init_zero_weights)

        # 2. Define the transformation part of the generator
        self.resnet_block  = ResnetBlock(conv_dim*2)

        # 3. Define the decoder part of the generator (that builds up the output image from features)
        self.upconv1 = upconv(in_channels=conv_dim*2, out_channels=conv_dim, kernel_size=5)
        self.upconv2 = upconv(in_channels=conv_dim, out_channels=3, kernel_size=5, batch_norm=False)

    def forward(self, x):
        """Generates an image conditioned on an input image.

            Input
            -----
                x: BS x 3 x 32 x 32

            Output
            ------
                out: BS x 3 x 32 x 32
        """
        batch_size = x.size(0)
        
        out = F.relu(self.conv1(x))            # BS x 32 x 16 x 16
        out = F.relu(self.conv2(out))          # BS x 64 x 8 x 8
        
        out = F.relu(self.resnet_block(out))   # BS x 64 x 8 x 8

        out = F.relu(self.upconv1(out))        # BS x 32 x 16 x 16

        out = F.tanh(self.upconv2(out))        # BS x 3 x 32 x 32
        
        out_size = out.size()
        if out_size != torch.Size([batch_size, 3, 32, 32]):
            raise ValueError("expect {} x 3 x 32 x 32, but get {}".format(batch_size, out_size))


        return out

### CycleGAN training loop

In [0]:
def cyclegan_training_loop(dataloader_X, dataloader_Y, test_dataloader_X, test_dataloader_Y, opts):
    """Runs the training loop.
        * Saves checkpoint every opts.checkpoint_every iterations
        * Saves generated samples every opts.sample_every iterations
    """

    # Create generators and discriminators
    G_XtoY, G_YtoX, D_X, D_Y = create_model(opts)

    g_params = list(G_XtoY.parameters()) + list(G_YtoX.parameters())  # Get generator parameters
    d_params = list(D_X.parameters()) + list(D_Y.parameters())  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = optim.Adam(d_params, opts.lr, [opts.beta1, opts.beta2])

    iter_X = iter(dataloader_X)
    iter_Y = iter(dataloader_Y)

    test_iter_X = iter(test_dataloader_X)
    test_iter_Y = iter(test_dataloader_Y)

    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_X = to_var(test_iter_X.next()[0])
    fixed_Y = to_var(test_iter_Y.next()[0])

    iter_per_epoch = min(len(iter_X), len(iter_Y))

    try:
        for iteration in range(1, opts.train_iters+1):

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
                iter_X = iter(dataloader_X)
                iter_Y = iter(dataloader_Y)

            images_X, labels_X = iter_X.next()
            images_X, labels_X = to_var(images_X), to_var(labels_X).long().squeeze()

            images_Y, labels_Y = iter_Y.next()
            images_Y, labels_Y = to_var(images_Y), to_var(labels_Y).long().squeeze()


            # ============================================
            #            TRAIN THE DISCRIMINATORS
            # ============================================

            #########################################
            ##             FILL THIS IN            ##
            #########################################

            # Train with real images
            d_optimizer.zero_grad()

            # 1. Compute the discriminator losses on real images
            # D_X_loss = ...
            # D_Y_loss = ...
            D_X_loss = torch.mean((D_X(images_X) - 1)**2)
            D_Y_loss = torch.mean((D_Y(images_Y) - 1)**2)

            d_real_loss = D_X_loss + D_Y_loss
            d_real_loss.backward()
            d_optimizer.step()

            # Train with fake images
            d_optimizer.zero_grad()

            # 2. Generate fake images that look like domain X based on real images in domain Y
            # fake_X = ...
            fake_X = G_YtoX(images_Y)

            # 3. Compute the loss for D_X
            # D_X_loss = ...
            D_X_loss = torch.mean(D_X(fake_X)**2)

            # 4. Generate fake images that look like domain Y based on real images in domain X
            # fake_Y = ...
            fake_Y = G_XtoY(images_X)

            # 5. Compute the loss for D_Y
            # D_Y_loss = ...
            D_Y_loss = torch.mean(D_Y(fake_Y)**2)

            d_fake_loss = D_X_loss + D_Y_loss
            d_fake_loss.backward()
            d_optimizer.step()



            # =========================================
            #            TRAIN THE GENERATORS
            # =========================================


            #########################################
            ##    FILL THIS IN: Y--X-->Y CYCLE     ##
            #########################################
            g_optimizer.zero_grad()

            # 1. Generate fake images that look like domain X based on real images in domain Y
            # fake_X = ...
            fake_X = G_YtoX(images_Y)

            # 2. Compute the generator loss based on domain X
            # g_loss = ...
            g_loss = torch.mean((D_X(fake_X) - 1)**2)

            reconstructed_Y = G_XtoY(fake_X)
            # 3. Compute the cycle consistency loss (the reconstruction loss)
            # cycle_consistency_loss = ...
            cycle_consistency_loss = torch.mean(torch.sum(torch.abs(images_Y - reconstructed_Y), [1,2,3]))

            g_loss += opts.lambda_cycle * cycle_consistency_loss

            g_loss.backward()
            g_optimizer.step()



            #########################################
            ##    FILL THIS IN: X--Y-->X CYCLE     ##
            #########################################

            g_optimizer.zero_grad()

            # 1. Generate fake images that look like domain Y based on real images in domain X
            # fake_Y = ...
            fake_Y = G_XtoY(images_X)

            # 2. Compute the generator loss based on domain Y
            # g_loss = ...
            g_loss = torch.mean((D_Y(fake_Y) - 1)**2)

            reconstructed_X = G_YtoX(fake_Y)
            # 3. Compute the cycle consistency loss (the reconstruction loss)
            # cycle_consistency_loss = ...
            cycle_consistency_loss = torch.mean(torch.sum(torch.abs(images_X - reconstructed_X),[1,2,3]))

            g_loss += opts.lambda_cycle * cycle_consistency_loss

            g_loss.backward()
            g_optimizer.step()


            # Print the log info
            if iteration % opts.log_step == 0:
                print('Iteration [{:5d}/{:5d}] | d_real_loss: {:6.4f} | d_Y_loss: {:6.4f} | d_X_loss: {:6.4f} | '
                    'd_fake_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                      iteration, opts.train_iters, d_real_loss.item(), D_Y_loss.item(),
                      D_X_loss.item(), d_fake_loss.item(), g_loss.item()))


            # Save the generated samples
            if iteration % opts.sample_every == 0:
                cyclegan_save_samples(iteration, fixed_Y, fixed_X, G_YtoX, G_XtoY, opts)


            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                cyclegan_checkpoint(iteration, G_XtoY, G_YtoX, D_X, D_Y, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G_XtoY, G_YtoX, D_X, D_Y

    return G_XtoY, G_YtoX, D_X, D_Y


# Training


## Download dataset

In [21]:
######################################################################
# Download Translation datasets
######################################################################
data_fpath = get_file(fname='emojis', 
                         origin='http://www.cs.toronto.edu/~jba/emojis.tar.gz', 
                         untar=True)

data/emojis.tar.gz


## DCGAN

In [37]:
SEED = 6

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':64,
              'noise_size':100,
              'num_workers': 0,
              'train_iters':20000,
              'X':'Windows',  # options: 'Windows' / 'Apple'
              'Y': None,
              'lr':0.0003,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_gan',
              'sample_dir': 'results/samples_gan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': True,
              'd_train_iters': 1
}
args.update(args_dict)

print_opts(args)
G, D = train(args)

generate_gif("results/samples_gan")

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 64                                     
                             noise_size: 100                                    
                            train_iters: 20000                                  
                                      X: Windows                                
                                     lr: 0.0003                                 
                                  beta1: 0.5                                    
                                  beta2: 0.999                                  
                             batch_size: 32                                     
                         che

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration [ 200/20000] | D_real_loss: 0.0168 | D_fake_loss: 0.0110 | G_loss: 0.9848
Saved results/samples_gan/sample-000200.png


Iteration [ 400/20000] | D_real_loss: 0.0959 | D_fake_loss: 0.0139 | G_loss: 0.7969
Saved results/samples_gan/sample-000400.png


Iteration [ 600/20000] | D_real_loss: 0.1199 | D_fake_loss: 0.0392 | G_loss: 0.4770
Saved results/samples_gan/sample-000600.png


...

Iteration [19200/20000] | D_real_loss: 0.0018 | D_fake_loss: 0.0004 | G_loss: 0.9607
Saved results/samples_gan/sample-019200.png


Iteration [19400/20000] | D_real_loss: 0.0013 | D_fake_loss: 0.0005 | G_loss: 0.9516
Saved results/samples_gan/sample-019400.png


Iteration [19600/20000] | D_real_loss: 0.0054 | D_fake_loss: 0.0033 | G_loss: 1.1376
Saved results/samples_gan/sample-019600.png


Iteration [19800/20000] | D_real_loss: 0.0015 | D_fake_loss: 0.0018 | G_loss: 0.9960
Saved results/samples_gan/sample-019800.png


Iteration [20000/20000] | D_real_loss: 0.0014 | D_fake_loss: 0.0015 | G_loss: 1.1446
Saved results/samples_gan/sample-020000.png
adding image results/samples_gan/losses.png
adding image results/samples_gan/sample-000200.png
adding image results/samples_gan/sample-000400.png
adding image results/samples_gan/sample-000600.png
adding image results/samples_gan/sample-000800.png
adding image results/samples_gan/sample-001000.png
adding image results/samples_gan/sample-001200.png
adding image results/samples_gan/sample-001400.png
adding image results/samples_gan/sample-001600.png
adding image results/samples_gan/sample-001800.png
adding image results/samples_gan/sample-002000.png
adding image results/samples_gan/sample-002200.png
adding image results/samples_gan/sample-002400.png
adding image results/samples_gan/sample-002600.png
adding image results/samples_gan/sample-002800.png
adding image results/samples_gan/sample-003000.png
adding image results/samples_gan/sample-003200.png
adding imag

## CycleGAN

In [36]:
SEED = 30

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':32,
              'init_zero_weights': False,
              'num_workers': 0,
              'train_iters':5000,
              'X':'Apple',
              'Y':'Windows',
              'lambda_cycle': 0.015,
              'lr':0.0003,
              'beta1':0.3,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_cyclegan',
              'sample_dir': 'results/samples_cyclegan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000
}
args.update(args_dict)


print_opts(args)
G_XtoY, G_YtoX, D_X, D_Y = train(args)
generate_gif("results/samples_cyclegan", keyword='X-Y')
generate_gif("results/samples_cyclegan", keyword='Y-X')

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 32                                     
                            train_iters: 5000                                   
                                      X: Apple                                  
                                      Y: Windows                                
                           lambda_cycle: 0.015                                  
                                     lr: 0.0003                                 
                                  beta1: 0.3                                    
                                  beta2: 0.999                                  
                            

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration [  200/ 5000] | d_real_loss: 0.0786 | d_Y_loss: 0.0329 | d_X_loss: 0.0622 | d_fake_loss: 0.0952 | g_loss: 8.3433
Saved results/samples_cyclegan/sample-000200-X-Y.png
Saved results/samples_cyclegan/sample-000200-Y-X.png


Iteration [  400/ 5000] | d_real_loss: 0.0501 | d_Y_loss: 0.0114 | d_X_loss: 0.0223 | d_fake_loss: 0.0337 | g_loss: 6.9063
Saved results/samples_cyclegan/sample-000400-X-Y.png
Saved results/samples_cyclegan/sample-000400-Y-X.png


Iteration [  600/ 5000] | d_real_loss: 0.1261 | d_Y_loss: 0.0319 | d_X_loss: 0.0265 | d_fake_loss: 0.0584 | g_loss: 6.9021
Saved results/samples_cyclegan/sample-000600-X-Y.png
Saved results/samples_cyclegan/sample-000600-Y-X.png


...

Iteration [ 4200/ 5000] | d_real_loss: 0.0288 | d_Y_loss: 0.0090 | d_X_loss: 0.0194 | d_fake_loss: 0.0284 | g_loss: 5.6035
Saved results/samples_cyclegan/sample-004200-X-Y.png
Saved results/samples_cyclegan/sample-004200-Y-X.png


Iteration [ 4400/ 5000] | d_real_loss: 0.0352 | d_Y_loss: 0.0158 | d_X_loss: 0.0498 | d_fake_loss: 0.0655 | g_loss: 6.0195
Saved results/samples_cyclegan/sample-004400-X-Y.png
Saved results/samples_cyclegan/sample-004400-Y-X.png


Iteration [ 4600/ 5000] | d_real_loss: 0.0334 | d_Y_loss: 0.0135 | d_X_loss: 0.0112 | d_fake_loss: 0.0247 | g_loss: 6.3619
Saved results/samples_cyclegan/sample-004600-X-Y.png
Saved results/samples_cyclegan/sample-004600-Y-X.png


Iteration [ 4800/ 5000] | d_real_loss: 0.0490 | d_Y_loss: 0.0089 | d_X_loss: 0.0120 | d_fake_loss: 0.0209 | g_loss: 5.8268
Saved results/samples_cyclegan/sample-004800-X-Y.png
Saved results/samples_cyclegan/sample-004800-Y-X.png


Iteration [ 5000/ 5000] | d_real_loss: 0.0992 | d_Y_loss: 0.0424 | d_X_loss: 0.0386 | d_fake_loss: 0.0810 | g_loss: 6.3783
Saved results/samples_cyclegan/sample-005000-X-Y.png
Saved results/samples_cyclegan/sample-005000-Y-X.png
adding image results/samples_cyclegan/sample-000200-X-Y.png
adding image results/samples_cyclegan/sample-000400-X-Y.png
adding image results/samples_cyclegan/sample-000600-X-Y.png
adding image results/samples_cyclegan/sample-000800-X-Y.png
adding image results/samples_cyclegan/sample-001000-X-Y.png
adding image results/samples_cyclegan/sample-001200-X-Y.png
adding image results/samples_cyclegan/sample-001400-X-Y.png
adding image results/samples_cyclegan/sample-001600-X-Y.png
adding image results/samples_cyclegan/sample-001800-X-Y.png
adding image results/samples_cyclegan/sample-002000-X-Y.png
adding image results/samples_cyclegan/sample-002200-X-Y.png
adding image results/samples_cyclegan/sample-002400-X-Y.png
adding image results/samples_cyclegan/sample-002600

'lambda_cycle' = 0

In [39]:
SEED = 30

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':32,
              'init_zero_weights': False,
              'num_workers': 0,
              'train_iters':5000,
              'X':'Apple',
              'Y':'Windows',
              'lambda_cycle': 0,
              'lr':0.0003,
              'beta1':0.3,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_cyclegan',
              'sample_dir': 'results/samples_cyclegan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000
}
args.update(args_dict)


print_opts(args)
G_XtoY, G_YtoX, D_X, D_Y = train(args)
generate_gif("results/samples_cyclegan", keyword='X-Y')
generate_gif("results/samples_cyclegan", keyword='Y-X')

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 32                                     
                            train_iters: 5000                                   
                                      X: Apple                                  
                                      Y: Windows                                
                                     lr: 0.0003                                 
                                  beta1: 0.3                                    
                                  beta2: 0.999                                  
                             batch_size: 32                                     
                         che

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration [  200/ 5000] | d_real_loss: 0.1925 | d_Y_loss: 0.1414 | d_X_loss: 0.0364 | d_fake_loss: 0.1779 | g_loss: 1.9750
Saved results/samples_cyclegan/sample-000200-X-Y.png
Saved results/samples_cyclegan/sample-000200-Y-X.png


Iteration [  400/ 5000] | d_real_loss: 0.1318 | d_Y_loss: 0.0538 | d_X_loss: 0.0651 | d_fake_loss: 0.1188 | g_loss: 1.3640
Saved results/samples_cyclegan/sample-000400-X-Y.png
Saved results/samples_cyclegan/sample-000400-Y-X.png


Iteration [  600/ 5000] | d_real_loss: 0.0970 | d_Y_loss: 0.0589 | d_X_loss: 0.0310 | d_fake_loss: 0.0899 | g_loss: 1.3175
Saved results/samples_cyclegan/sample-000600-X-Y.png
Saved results/samples_cyclegan/sample-000600-Y-X.png


...

Iteration [ 4200/ 5000] | d_real_loss: 0.0455 | d_Y_loss: 0.0261 | d_X_loss: 0.0314 | d_fake_loss: 0.0575 | g_loss: 1.0777
Saved results/samples_cyclegan/sample-004200-X-Y.png
Saved results/samples_cyclegan/sample-004200-Y-X.png


Iteration [ 4400/ 5000] | d_real_loss: 0.0864 | d_Y_loss: 0.0448 | d_X_loss: 0.0706 | d_fake_loss: 0.1154 | g_loss: 1.2976
Saved results/samples_cyclegan/sample-004400-X-Y.png
Saved results/samples_cyclegan/sample-004400-Y-X.png


Iteration [ 4600/ 5000] | d_real_loss: 0.0637 | d_Y_loss: 0.0261 | d_X_loss: 0.0417 | d_fake_loss: 0.0678 | g_loss: 1.1837
Saved results/samples_cyclegan/sample-004600-X-Y.png
Saved results/samples_cyclegan/sample-004600-Y-X.png


Iteration [ 4800/ 5000] | d_real_loss: 0.0501 | d_Y_loss: 0.0368 | d_X_loss: 0.0196 | d_fake_loss: 0.0564 | g_loss: 0.8300
Saved results/samples_cyclegan/sample-004800-X-Y.png
Saved results/samples_cyclegan/sample-004800-Y-X.png


Iteration [ 5000/ 5000] | d_real_loss: 0.0553 | d_Y_loss: 0.0193 | d_X_loss: 0.0310 | d_fake_loss: 0.0503 | g_loss: 1.0897
Saved results/samples_cyclegan/sample-005000-X-Y.png
Saved results/samples_cyclegan/sample-005000-Y-X.png
adding image results/samples_cyclegan/sample-000200-X-Y.png
adding image results/samples_cyclegan/sample-000400-X-Y.png
adding image results/samples_cyclegan/sample-000600-X-Y.png
adding image results/samples_cyclegan/sample-000800-X-Y.png
adding image results/samples_cyclegan/sample-001000-X-Y.png
adding image results/samples_cyclegan/sample-001200-X-Y.png
adding image results/samples_cyclegan/sample-001400-X-Y.png
adding image results/samples_cyclegan/sample-001600-X-Y.png
adding image results/samples_cyclegan/sample-001800-X-Y.png
adding image results/samples_cyclegan/sample-002000-X-Y.png
adding image results/samples_cyclegan/sample-002200-X-Y.png
adding image results/samples_cyclegan/sample-002400-X-Y.png
adding image results/samples_cyclegan/sample-002600


'lambda_cycle' = 1






In [41]:
SEED = 30

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':32,
              'init_zero_weights': False,
              'num_workers': 0,
              'train_iters':5000,
              'X':'Apple',
              'Y':'Windows',
              'lambda_cycle': 1,
              'lr':0.0003,
              'beta1':0.3,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_cyclegan',
              'sample_dir': 'results/samples_cyclegan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000
}
args.update(args_dict)


print_opts(args)
G_XtoY, G_YtoX, D_X, D_Y = train(args)
generate_gif("results/samples_cyclegan", keyword='X-Y')
generate_gif("results/samples_cyclegan", keyword='Y-X')

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 32                                     
                            train_iters: 5000                                   
                                      X: Apple                                  
                                      Y: Windows                                
                           lambda_cycle: 1                                      
                                     lr: 0.0003                                 
                                  beta1: 0.3                                    
                                  beta2: 0.999                                  
                            

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration [  200/ 5000] | d_real_loss: 0.0746 | d_Y_loss: 0.0102 | d_X_loss: 0.0112 | d_fake_loss: 0.0214 | g_loss: 434.7769
Saved results/samples_cyclegan/sample-000200-X-Y.png
Saved results/samples_cyclegan/sample-000200-Y-X.png


Iteration [  400/ 5000] | d_real_loss: 0.0066 | d_Y_loss: 0.0074 | d_X_loss: 0.0116 | d_fake_loss: 0.0191 | g_loss: 345.1957
Saved results/samples_cyclegan/sample-000400-X-Y.png
Saved results/samples_cyclegan/sample-000400-Y-X.png


Iteration [  600/ 5000] | d_real_loss: 0.0070 | d_Y_loss: 0.0057 | d_X_loss: 0.0056 | d_fake_loss: 0.0113 | g_loss: 311.2488
Saved results/samples_cyclegan/sample-000600-X-Y.png
Saved results/samples_cyclegan/sample-000600-Y-X.png


...

Iteration [ 4200/ 5000] | d_real_loss: 0.0046 | d_Y_loss: 0.0031 | d_X_loss: 0.0003 | d_fake_loss: 0.0035 | g_loss: 219.9838
Saved results/samples_cyclegan/sample-004200-X-Y.png
Saved results/samples_cyclegan/sample-004200-Y-X.png


Iteration [ 4400/ 5000] | d_real_loss: 0.0028 | d_Y_loss: 0.0033 | d_X_loss: 0.0062 | d_fake_loss: 0.0096 | g_loss: 227.1474
Saved results/samples_cyclegan/sample-004400-X-Y.png
Saved results/samples_cyclegan/sample-004400-Y-X.png


Iteration [ 4600/ 5000] | d_real_loss: 0.0028 | d_Y_loss: 0.0049 | d_X_loss: 0.0030 | d_fake_loss: 0.0079 | g_loss: 257.9308
Saved results/samples_cyclegan/sample-004600-X-Y.png
Saved results/samples_cyclegan/sample-004600-Y-X.png


Iteration [ 4800/ 5000] | d_real_loss: 0.0019 | d_Y_loss: 0.0003 | d_X_loss: 0.0037 | d_fake_loss: 0.0040 | g_loss: 249.4363
Saved results/samples_cyclegan/sample-004800-X-Y.png
Saved results/samples_cyclegan/sample-004800-Y-X.png


Iteration [ 5000/ 5000] | d_real_loss: 0.0044 | d_Y_loss: 0.0043 | d_X_loss: 0.0122 | d_fake_loss: 0.0165 | g_loss: 230.6954
Saved results/samples_cyclegan/sample-005000-X-Y.png
Saved results/samples_cyclegan/sample-005000-Y-X.png
adding image results/samples_cyclegan/sample-000200-X-Y.png
adding image results/samples_cyclegan/sample-000400-X-Y.png
adding image results/samples_cyclegan/sample-000600-X-Y.png
adding image results/samples_cyclegan/sample-000800-X-Y.png
adding image results/samples_cyclegan/sample-001000-X-Y.png
adding image results/samples_cyclegan/sample-001200-X-Y.png
adding image results/samples_cyclegan/sample-001400-X-Y.png
adding image results/samples_cyclegan/sample-001600-X-Y.png
adding image results/samples_cyclegan/sample-001800-X-Y.png
adding image results/samples_cyclegan/sample-002000-X-Y.png
adding image results/samples_cyclegan/sample-002200-X-Y.png
adding image results/samples_cyclegan/sample-002400-X-Y.png
adding image results/samples_cyclegan/sample-0026

'lambda_cycle' = 10

In [43]:
SEED = 30

# Set the random seed manually for reproducibility.
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


args = AttrDict()
args_dict = {
              'image_size':32, 
              'g_conv_dim':32, 
              'd_conv_dim':32,
              'init_zero_weights': False,
              'num_workers': 0,
              'train_iters':5000,
              'X':'Apple',
              'Y':'Windows',
              'lambda_cycle': 10,
              'lr':0.0003,
              'beta1':0.3,
              'beta2':0.999,
              'batch_size':32, 
              'checkpoint_dir': 'results/checkpoints_cyclegan',
              'sample_dir': 'results/samples_cyclegan',
              'load': None,
              'log_step':200,
              'sample_every':200,
              'checkpoint_every':1000
}
args.update(args_dict)


print_opts(args)
G_XtoY, G_YtoX, D_X, D_Y = train(args)
generate_gif("results/samples_cyclegan", keyword='X-Y')
generate_gif("results/samples_cyclegan", keyword='Y-X')

                                      Opts                                      
--------------------------------------------------------------------------------
                             image_size: 32                                     
                             g_conv_dim: 32                                     
                             d_conv_dim: 32                                     
                            train_iters: 5000                                   
                                      X: Apple                                  
                                      Y: Windows                                
                           lambda_cycle: 10                                     
                                     lr: 0.0003                                 
                                  beta1: 0.3                                    
                                  beta2: 0.999                                  
                            

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration [  200/ 5000] | d_real_loss: 0.0652 | d_Y_loss: 0.0159 | d_X_loss: 0.0137 | d_fake_loss: 0.0296 | g_loss: 4359.2227
Saved results/samples_cyclegan/sample-000200-X-Y.png
Saved results/samples_cyclegan/sample-000200-Y-X.png


Iteration [  400/ 5000] | d_real_loss: 0.0065 | d_Y_loss: 0.0050 | d_X_loss: 0.0109 | d_fake_loss: 0.0159 | g_loss: 3472.7688
Saved results/samples_cyclegan/sample-000400-X-Y.png
Saved results/samples_cyclegan/sample-000400-Y-X.png


Iteration [  600/ 5000] | d_real_loss: 0.0087 | d_Y_loss: 0.0046 | d_X_loss: 0.0041 | d_fake_loss: 0.0087 | g_loss: 3127.4736
Saved results/samples_cyclegan/sample-000600-X-Y.png
Saved results/samples_cyclegan/sample-000600-Y-X.png


...

Iteration [ 4200/ 5000] | d_real_loss: 0.0045 | d_Y_loss: 0.0097 | d_X_loss: 0.0003 | d_fake_loss: 0.0101 | g_loss: 2192.8096
Saved results/samples_cyclegan/sample-004200-X-Y.png
Saved results/samples_cyclegan/sample-004200-Y-X.png


Iteration [ 4400/ 5000] | d_real_loss: 0.0022 | d_Y_loss: 0.0048 | d_X_loss: 0.0031 | d_fake_loss: 0.0079 | g_loss: 2217.8579
Saved results/samples_cyclegan/sample-004400-X-Y.png
Saved results/samples_cyclegan/sample-004400-Y-X.png


Iteration [ 4600/ 5000] | d_real_loss: 0.0009 | d_Y_loss: 0.0025 | d_X_loss: 0.0035 | d_fake_loss: 0.0061 | g_loss: 2574.7144
Saved results/samples_cyclegan/sample-004600-X-Y.png
Saved results/samples_cyclegan/sample-004600-Y-X.png


Iteration [ 4800/ 5000] | d_real_loss: 0.0061 | d_Y_loss: 0.0002 | d_X_loss: 0.0022 | d_fake_loss: 0.0024 | g_loss: 2465.5940
Saved results/samples_cyclegan/sample-004800-X-Y.png
Saved results/samples_cyclegan/sample-004800-Y-X.png


Iteration [ 5000/ 5000] | d_real_loss: 0.0041 | d_Y_loss: 0.0022 | d_X_loss: 0.0084 | d_fake_loss: 0.0106 | g_loss: 2315.5229
Saved results/samples_cyclegan/sample-005000-X-Y.png
Saved results/samples_cyclegan/sample-005000-Y-X.png
adding image results/samples_cyclegan/sample-000200-X-Y.png
adding image results/samples_cyclegan/sample-000400-X-Y.png
adding image results/samples_cyclegan/sample-000600-X-Y.png
adding image results/samples_cyclegan/sample-000800-X-Y.png
adding image results/samples_cyclegan/sample-001000-X-Y.png
adding image results/samples_cyclegan/sample-001200-X-Y.png
adding image results/samples_cyclegan/sample-001400-X-Y.png
adding image results/samples_cyclegan/sample-001600-X-Y.png
adding image results/samples_cyclegan/sample-001800-X-Y.png
adding image results/samples_cyclegan/sample-002000-X-Y.png
adding image results/samples_cyclegan/sample-002200-X-Y.png
adding image results/samples_cyclegan/sample-002400-X-Y.png
adding image results/samples_cyclegan/sample-002